In [0]:
!pip3 install -q robobrowser
import re
from robobrowser import RoboBrowser
from IPython.display import Image, HTML, display
from collections import namedtuple
import json
import os

In [3]:
browser = RoboBrowser(history=True,)
link = 'https://www.casarosada.gob.ar/nuestro-pais/galeria-de-presidentes'
browser.open(link)

llamados = browser.find_all(href=re.compile(r'Enviar',re.I))

pat = re.compile(r'.*..\/(.*)\?.*') 

x = ( llamado.get('href') for llamado in llamados )
y = ( pat.match(link) for link in x )
paginas = set( text.group(1) for text in y )



def obtener_header(text):
    pat = re.compile(r'(.*)\s+\((\w+) .*? (\w+)\)|(.*)\s+\((\w+)\)|(.*)\s+\((.+)-(.*)\)') 
    return tuple(s for s in pat.search(text).groups() if s is not None)

def obtener_cuerpo(pagina):
  OBRA = 'OBRA DE GOBIERNO'
  MINISTROS = 'MINIST'
  VICE='Vice'
  MANDATO='Mandato'
  keys = ('intro','obra','ministros','vicepresidente','mandato')
  key = keys[0]
  cuerpo = {key:[] for key in keys}
  cuerpo.update({keys[2]:{}})

  for x in pagina[0].find_all('p'):
      line = x.text.strip().replace(u'\xa0',' ')
      if len(line) == 0:
        continue 

      if line.startswith(OBRA):
        key = keys[1]
        continue
      elif line.startswith(MINISTROS):
        key = keys[2]
        continue
      elif line.startswith(VICE):
        cuerpo[keys[3]]=line.split(':',1)[1].strip()
        continue
      elif line.startswith(MANDATO):
        cuerpo[keys[4]]=line.split(':',1)[1].strip()
        continue  
      elif line.startswith('Información provista'):
        continue
        
      if key == keys[2]:
        try:
          m,n = line.split(':',1)
          cuerpo[key][m] = n.strip()
        except:
          cuerpo[key]['indefinido'] = line.strip()
          print(line)
      else:  
        cuerpo[key].append(line)
 
  return cuerpo

/usr/local/lib/python3.6/dist-packages/robobrowser/browser.py:40: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 40 of the file /usr/local/lib/python3.6/dist-packages/robobrowser/browser.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  features=self.browser.parser,


In [10]:
texto_paginas = []
for i,pagina in enumerate(paginas):
  browser.open(f'{link}/{pagina}')
  datos = {}
  try:
    p =  browser.select('div.item-page')
    datos['titulo'] = obtener_header(p[0].find('a').text.strip())
    datos.update(obtener_cuerpo(p))
  except:
    p = browser.select('div.jumbotron-hero')
    datos['titulo'] = obtener_header(p[0].find('h2').text.strip())
    p =  browser.select('div.item-page')
    datos.update(obtener_cuerpo(p))
    
  texto_paginas.append(datos)
  
texto_paginas[0]

/usr/local/lib/python3.6/dist-packages/robobrowser/browser.py:40: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 40 of the file /usr/local/lib/python3.6/dist-packages/robobrowser/browser.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  features=self.browser.parser,


Luis L. Domínguez. Santiago Cortínez.


{'intro': ['Raúl Alberto Lastiri',
  'Nace en Buenos Aires, el 11 de septiembre de 1915. Fallece en Buenos Aires, el 11 de diciembre de 1978.',
  'Casado con Amelia Concepción Martino, tras divorciarse forma pareja con Norma López Rega.',
  'Presidente de la Nación Argentina (interino). Asume por ser Presidente de la Cámara de',
  'Diputados, ante las renuncias de Héctor Cámpora y Vicente Solano Lima (Presidente y Vice,',
  'respectivamente) y la ausencia de Alejandro Díaz Bialet (Vicepresidente Provisional del Senado)'],
 'mandato': '13 de julio de 1973 al 12 de octubre de 1973.',
 'ministros': {'Bienestar Social': 'José López Rega.',
  'Cultura y Educación': 'Jorge Alberto Taiana.',
  'Defensa Nacional': 'Angel Federico Robledo.',
  'Economía y Finanzas': 'José Ber Gelbard.',
  'Interior': 'Benito Llambí.',
  'Justicia': 'Antonio J. Benítez. Trabajo: Ricardo Otero.',
  'RR.EE. y Culto': 'Juan Alberto Vignes.'},
 'obra': ['Anuncio de la preparación de un Plan Trienal de desarrollo, de

In [0]:
with open('presidentes.json','w',encoding='utf8') as f:
      f.write(json.dumps(texto_paginas,ensure_ascii=False))

In [16]:
!cat presidentes.json

[{"titulo": ["RAÚL ALBERTO LASTIRI", "1973", "1973"], "intro": ["Raúl Alberto Lastiri", "Nace en Buenos Aires, el 11 de septiembre de 1915. Fallece en Buenos Aires, el 11 de diciembre de 1978.", "Casado con Amelia Concepción Martino, tras divorciarse forma pareja con Norma López Rega.", "Presidente de la Nación Argentina (interino). Asume por ser Presidente de la Cámara de", "Diputados, ante las renuncias de Héctor Cámpora y Vicente Solano Lima (Presidente y Vice,", "respectivamente) y la ausencia de Alejandro Díaz Bialet (Vicepresidente Provisional del Senado)"], "obra": ["Anuncio de la preparación de un Plan Trienal de desarrollo, de 1974 a 1976.", "Freno a las amnistías de presos políticos y decreto que declara ilegal a la organización guerrillera Ejército Revolucionario del Pueblo.", "Convocatoria a Elecciones para el 23 de septiembre."], "ministros": {"Interior": "Benito Llambí.", "RR.EE. y Culto": "Juan Alberto Vignes.", "Economía y Finanzas": "José Ber Gelbard.", "Cultura y Educ

In [13]:
texto_paginas

[{'intro': ['Raúl Alberto Lastiri',
   'Nace en Buenos Aires, el 11 de septiembre de 1915. Fallece en Buenos Aires, el 11 de diciembre de 1978.',
   'Casado con Amelia Concepción Martino, tras divorciarse forma pareja con Norma López Rega.',
   'Presidente de la Nación Argentina (interino). Asume por ser Presidente de la Cámara de',
   'Diputados, ante las renuncias de Héctor Cámpora y Vicente Solano Lima (Presidente y Vice,',
   'respectivamente) y la ausencia de Alejandro Díaz Bialet (Vicepresidente Provisional del Senado)'],
  'mandato': '13 de julio de 1973 al 12 de octubre de 1973.',
  'ministros': {'Bienestar Social': 'José López Rega.',
   'Cultura y Educación': 'Jorge Alberto Taiana.',
   'Defensa Nacional': 'Angel Federico Robledo.',
   'Economía y Finanzas': 'José Ber Gelbard.',
   'Interior': 'Benito Llambí.',
   'Justicia': 'Antonio J. Benítez. Trabajo: Ricardo Otero.',
   'RR.EE. y Culto': 'Juan Alberto Vignes.'},
  'obra': ['Anuncio de la preparación de un Plan Trienal de